In [16]:
#import required libraries

import pandas as pd
import yfinance as yf
import numpy as np
import math
import pandas_ta as ta

In [7]:
# parameter setup (default values in the original indicator)
length = 20
mult = 2
length_KC = 21
mult_KC = 1.5

In [8]:
stock = yf.Ticker("AAPL")
print(stock.info)

{'zip': '95014', 'sector': 'Technology', 'fullTimeEmployees': 100000, 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radi

In [9]:
stock_df = stock.history(period="1y", interval='1d')
stock_df.index.duplicated().sum()
df_stock = stock_df.loc[~stock_df.index.duplicated(keep='first')]
df_stock

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-28,93.735900,93.917181,92.623393,92.628365,103625600,0.0,0.0
2020-07-29,93.122526,94.592623,93.085278,94.403893,90329200,0.0,0.0
2020-07-30,93.557104,95.652982,93.139917,95.546204,158130000,0.0,0.0
2020-07-31,102.196394,105.702767,100.150176,105.548805,374336800,0.0,0.0
2020-08-03,107.475822,110.890315,107.170385,108.208389,308151200,0.0,0.0
...,...,...,...,...,...,...,...
2021-07-21,145.529999,146.130005,144.630005,145.399994,74915000,0.0,0.0
2021-07-22,145.940002,148.199997,145.809998,146.800003,77338200,0.0,0.0
2021-07-23,147.550003,148.720001,146.919998,148.559998,71361600,0.0,0.0


In [10]:
# calculate Bollinger Bands
# moving average
m_avg = df_stock['Close'].rolling(window=length).mean()
# standard deviation
m_std = df_stock['Close'].rolling(window=length).std(ddof=0)
# upper Bollinger Bands
df_stock['upper_BB'] = m_avg + mult * m_std
# lower Bollinger Bands 
df_stock['lower_BB'] = m_avg - mult * m_std

In [11]:
# calculate Keltner Channel
# first we need to calculate True Range
df_stock['tr0'] = abs(df_stock["High"] - df_stock["Low"])
df_stock['tr1'] = abs(df_stock["High"] - df_stock["Close"].shift())
df_stock['tr2'] = abs(df_stock["Low"] - df_stock["Close"].shift())
df_stock['tr'] = df_stock[['tr0', 'tr1', 'tr2']].max(axis=1)
# moving average of the TR
range_ma = df_stock['tr'].rolling(window=length_KC).mean()
# upper Keltner Channel
df_stock['upper_KC'] = m_avg + range_ma * mult_KC
# lower Keltner Channel
df_stock['lower_KC'] = m_avg - range_ma * mult_KC

In [19]:
# check for 'squeeze'
df_stock['squeeze_on'] = (df_stock['lower_BB'] > df_stock['lower_KC']) | (df_stock['upper_BB'] < df_stock['upper_KC'])
df_stock['squeeze_off'] = (df_stock['lower_BB'] < df_stock['lower_KC']) | (df_stock['upper_BB'] > df_stock['upper_KC'])

In [37]:
# Momentum
highest = df_stock['High'].rolling(window = length_KC).max()
lowest = df_stock['Low'].rolling(window = length_KC).min()
m1 = (highest + lowest) / 2
df_stock['value'] = (df_stock['Close'] - (m1 + m_avg)/2)
fit_y = np.array(range(0,length_KC))
df_stock['value'] = df_stock['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
   np.polyfit(fit_y, x, 1)[1], raw=True)

df_stock.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,upper_BB,lower_BB,tr0,tr1,tr2,tr,upper_KC,lower_KC,squeeze_on,squeeze_off,value
Date,,,,,,,,,,,,,,,,,,
2020-07-28,93.735900,93.917181,92.623393,92.628365,103625600,0.0,0.0,NaN,NaN,1.293788,NaN,NaN,1.293788,NaN,NaN,False,False,NaN
2020-07-29,93.122526,94.592623,93.085278,94.403893,90329200,0.0,0.0,NaN,NaN,1.507345,1.964259,0.456914,1.964259,NaN,NaN,False,False,NaN
2020-07-30,93.557104,95.652982,93.139917,95.546204,158130000,0.0,0.0,NaN,NaN,2.513066,1.249090,1.263976,2.513066,NaN,NaN,False,False,NaN
2020-07-31,102.196394,105.702767,100.150176,105.548805,374336800,0.0,0.0,NaN,NaN,5.552590,10.156563,4.603973,10.156563,NaN,NaN,False,False,NaN
2020-08-03,107.475822,110.890315,107.170385,108.208389,308151200,0.0,0.0,NaN,NaN,3.719930,5.341510,1.621580,5.341510,NaN,NaN,False,False,NaN


In [23]:
# entry point for long position:
# 1. black cross becomes gray (the squeeze is released)
long_cond1 = (df_stock['squeeze_off'][-2] == False) | (df_stock['squeeze_off'][-1] == True) 
# 2. bar value is positive => the bar is light green
long_cond2 = df_stock['value'][-1] > 0

enter_long = long_cond1 and long_cond2
# entry point for short position:
# 1. black cross becomes gray (the squeeze is released)
short_cond1 = (df_stock['squeeze_off'][-2] == False) | (df_stock['squeeze_off'][-1] == True) 
# 2. bar value is negative => the bar is light red 
short_cond2 = df_stock['value'][-1] < 0
enter_short = short_cond1 and short_cond2

In [46]:
import mplfinance as mpf
%matplotlib widget

In [39]:
ohcl = df_stock[['Open', 'High', 'Close', 'Low']]
ohcl.head()

,Open,High,Close,Low
Date,,,,
2020-07-28,93.735900,93.917181,92.628365,92.623393
2020-07-29,93.122526,94.592623,94.403893,93.085278
2020-07-30,93.557104,95.652982,95.546204,93.139917
2020-07-31,102.196394,105.702767,105.548805,100.150176
2020-08-03,107.475822,110.890315,108.208389,107.170385


In [40]:
# add colors for the 'value bar'
colors = []
for ind, val in enumerate(df_stock['value']):
  if val >= 0:
    color = 'green'
    if val > df_stock['value'][ind-1]:
      color = 'lime'
  else:
    color = 'maroon'
    if val < df_stock['value'][ind-1]:
      color='red'
  colors.append(color)

In [43]:
# add 2 subplots: 1. bars, 2. crosses
apds = [mpf.make_addplot(df_stock['value'], panel=1, type='bar', color=colors, alpha=0.8, secondary_y=False),
        mpf.make_addplot([0] * len(df_stock), panel=1, type='scatter', marker='x', markersize=50, color=['gray' if s else 'black' for s in df_stock['squeeze_off']], secondary_y=False)]

# plot ohcl with subplots
fig, axes = mpf.plot(ohcl, 
              volume_panel = 2,
              figratio=(2,1),
              figscale=1,
              mav = (8,21,34),
              type='candle', 
              addplot=apds,
              returnfig=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
screened_list = []
stock_list = ['AAPL','TSLA','MSFT','AMZN']

for stock_code in stock_list:
    df = yf.download(stock_code, start='2020-01-01', threads= False)
    if enter_long | enter_short:
        screened_list.append(stock_code)
    
if screened_list:
  print(screened_list)
else:
  print('No stock fits the indicator entry requirement')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
['AAPL', 'TSLA', 'MSFT', 'AMZN']


In [44]:
help(mpf.plot)

Help on function plot in module mplfinance.plotting:

plot(data, **kwargs)
    Given a Pandas DataFrame containing columns Open,High,Low,Close and optionally Volume
    with a DatetimeIndex, plot the data.
    Available plots include ohlc bars, candlestick, and line plots.
    Also provide visually analysis in the form of common technical studies, such as:
    moving averages, renko, etc.
    Also provide ability to plot trading signals, and/or addtional user-defined data.

